###### Simple multithreading

In [63]:
from time import sleep
from concurrent import futures
from random import randint

In [64]:
dat = [randint(1, 9) for x in range(10)]

In [65]:
def task(time_to_sleep):
    sleep(time_to_sleep)
    print('sleepped for', time_to_sleep)
    return time_to_sleep

In [66]:
with futures.ThreadPoolExecutor(max_workers=10) as executor:
    to_do = []
    for d in dat:
        to_do.append(executor.submit(task, d))
    
    for x in futures.as_completed(to_do):
        print(x.result())

sleepped forsleepped forsleepped forsleepped for 2 2
 2
2

2
2
 2
2
sleepped for 4
4
sleepped forsleepped for 6 6

6
6
sleepped forsleepped for 7 7

7
7
sleepped for 8
8


###### With timeout

- https://www.reddit.com/r/Python/comments/3de2lz/force_thread_to_timeout_with_threadpoolexecutor/
- https://stackoverflow.com/questions/6509261/how-to-use-concurrent-futures-with-timeouts
- https://codereview.stackexchange.com/questions/142828/python-executer-that-kills-processes-after-a-timeout

In [14]:
from time import sleep
from concurrent import futures
from random import randint
from datetime import datetime

dat = [randint(1, 20) for x in range(10)]

def task(time_to_sleep):
    sleep(1)
    print(datetime.now(), 'task called with', time_to_sleep)
    sleep(time_to_sleep)
    print(datetime.now(), 'awake after', time_to_sleep)
    return time_to_sleep

print(datetime.now(), 'started for', dat)
with futures.ThreadPoolExecutor(max_workers=100) as executor:
    to_do = {}
    result = []
    
    for d in dat:
        to_do[executor.submit(task, d)] = d

    print(datetime.now(), 'submition done')
    
    try:
        print(datetime.now(), 'before as_complted')
        for x in futures.as_completed(to_do, timeout=10):
            print(datetime.now(), 'return', x.result())
            result.append(x.result())
    except futures.TimeoutError as e:
        print(datetime.now(), 'in except')
        print(e)
    

2019-01-29 08:35:44.803560 started for [12, 10, 8, 4, 8, 8, 6, 3, 19, 15]
2019-01-29 08:35:44.808566 submition done
2019-01-29 08:35:44.808566 before as_complted
2019-01-29 08:35:45.8055592019-01-29 08:35:45.8165632019-01-29 08:35:45.8275782019-01-29 08:35:45.838563 task called with 2019-01-29 08:35:45.850562  task called with 8
2019-01-29 08:35:45.860566task called with 4
122019-01-29 08:35:45.868558 task called with 8 task called with 3
2019-01-29 08:35:45.876562
 task called with task called with  8
2019-01-29 08:35:45.885561 task called with 62019-01-29 08:35:45.894578

19
 task called with 10
 task called with 15
2019-01-29 08:35:48.877559 awake after 3
2019-01-29 08:35:48.877559 return 3
2019-01-29 08:35:49.869617 awake after 4
2019-01-29 08:35:49.869617 return 4
2019-01-29 08:35:51.902612 awake after 6
2019-01-29 08:35:51.902612 return 6
2019-01-29 08:35:53.861559 awake after 8
2019-01-29 08:35:53.861559 return 8
2019-01-29 08:35:53.885594 awake after 8
2019-01-29 08:35:53.88559

###### timeout with Pebble

- multiprocess examples below do not work in the notebook

In [40]:
from pebble import concurrent

@concurrent.thread
def function(foo, bar=0):
    return foo + bar

future = function(1, bar=2)

result = future.result()  # blocks until results are ready

result

3

In [7]:
from concurrent.futures import TimeoutError
from pebble import ProcessPool, ProcessExpired
import time
from random import randint
from datetime import datetime

dat = [randint(1, 10) for x in range(10)]


def task(time_to_sleep):
    time.sleep(1)
    print(datetime.now(), 'task called with', time_to_sleep)
    time.sleep(time_to_sleep)
    print(datetime.now(), 'awake after', time_to_sleep)
    return time_to_sleep


if __name__ == '__main__':
    print(datetime.now(), 'started for', dat)
    with ProcessPool() as pool:
        future = pool.map(task, dat, timeout=3)
        iterator = future.result()

        while True:
            try:
                result = next(iterator)
                print(result)
            except StopIteration:
                break
            except TimeoutError as error:
                print("function took longer than %d seconds" % error.args[1])
            except ProcessExpired as error:
                print("%s. Exit code: %d" % (error, error.exitcode))
            except Exception as error:
                print("function raised %s" % error)
                print(error.traceback)  # Python's traceback of remote process


2019-01-29 10:03:44.494673 started for [1, 4, 3, 5, 5, 3, 7, 8, 9, 3]


In [5]:
from concurrent.futures import TimeoutError
from pebble import ProcessPool, ProcessExpired
import time
from random import randint
from datetime import datetime

dat = [randint(1, 10) for x in range(10)]


def task(time_to_sleep):
    time.sleep(1)
    print(datetime.now(), 'task called with', time_to_sleep)
    time.sleep(time_to_sleep)
    print(datetime.now(), 'awake after', time_to_sleep)
    return time_to_sleep


if __name__ == '__main__':
    print(datetime.now(), 'started for', dat)
    to_do = []
    with ProcessPool() as pool:
        for d in dat:
            future = pool.schedule(task, [d], timeout=5)
            to_do.append(future)

        for td in to_do:
            try:
                r = td.result()
                print(r)
            except TimeoutError as error:
                print("function took longer than %d seconds" % error.args[1])
            except ProcessExpired as error:
                print("%s. Exit code: %d" % (error, error.exitcode))
            except Exception as error:
                print("function raised %s" % error)
                print(error.traceback)  # Python's traceback of remote process



[4, 6, 7, 7, 4, 4, 9, 4, 8, 8]

In [ ]:
from concurrent.futures import TimeoutError
from pebble import ProcessPool, ProcessExpired
import time
from random import randint
from datetime import datetime
import multiprocessing


dat = [randint(1, 10) for x in range(10)]


def task(time_to_sleep):
    print(datetime.now(), 'task called with', time_to_sleep)
    time.sleep(time_to_sleep)
    print(datetime.now(), 'awake after', time_to_sleep)
    return time_to_sleep


num_cpu = multiprocessing.cpu_count()
if __name__ == '__main__':
    print(datetime.now(), 'started for', dat)
    to_do = {}
    with ProcessPool() as pool:
        print(datetime.now(), 'submitting')
        for d in dat:
            future = pool.schedule(task, [d], timeout=5)
            to_do[future] = d

        print(datetime.now(), 'joining')
        for td, val in to_do.items():
            try:
                r = td.result()
                print(r)
            except TimeoutError as error:
                print("function took longer than %d seconds" % error.args[1], val)
            except ProcessExpired as error:
                print("%s. Exit code: %d" % (error, error.exitcode), val)
            except Exception as error:
                print("function raised %s" % error, val)
                print(error.traceback)  # Python's traceback of remote process




###### asyncio

In [1]:
import asyncio

import sys
import itertools

write = sys.stdout.write
flush = sys.stdout.flush

i = 0
msg = 'test'
for char in itertools.cycle('|/-\\'):
    i += 1
    
    status = char + ' ' + msg
    write(status)
    flush()
    write('\x08' * len(status))
    
    if i > 10:
        break